# <strong>SQL Injection</strong>

SQL (Structured Query Language) injection (abbreviated as SQLi) is a type of database attack by typing SQL code into an input field. SQL injection attacks can be very detrimental, as they can be used to leak information, change entries, wipe entire databases, and more. For this lab, you will be experimenting with a bank website that contains insecure code that does not sanitize user input, opening it up to security vulnerabilities.

<figure><center><img src="resources/sqli/sqli_example.jpg" style="width: 95%; height: 85%;"></img></center><em><figcaption>Credit: <a href="https://www.dnsstuff.com/sql-injection">DNSstuff</a></figcaption></em></figure>

The website where you will practice SQL injection is developed in PHP, which is hosted through Apache. PHP is a server-side language that's used for handling and processing data for websites. Since PHP is server-side, its code cannot be viewed by the user. However, this does not prevent attackers from attemping SQL injection, and you will learn in this lab how hackers can guess-and-check various SQL statements to crack a database.

<strong>This lab will contain four topics, and you will learn the following:</strong>

1. SQL Basics
2. PHP and SQL Practice
3. Prepared Statements
4. A Large-Scale Application of SQL Injection

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/save.py sqli"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/load.py sqli"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Begin the experiment.

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
def startlab(button):
    # Defining the lab name.
    labname = "sqli"

    # Writing the information to an empty field below the button.
    with output0:
        output0.clear_output()
        
        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + ".umdsec[a-z]{1,2}"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - umdsectc -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing materialization for this lab already exists. </span><span>You might have ran another \
            lab without stopping this one. Attaching the existing materialization.</span>"))
            subprocess.run(f'su - umdsectc -c "mrg xdc attach xdc '+ match.group(0) + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing materializations are found.</span>"))
        
            # Second, start the lab.
            display(HTML("<span>Starting " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            startexp = subprocess.run('su - umdsectc -c "bash /share/startexp ' + labname + '"', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout)

            # Another lab is already attached to the XDC.
            if (("XDC already attached") in startexp.stdout):
                existingLab = re.search(r"real.(.*).umdsec[a-z]{1,2}", startexp.stdout).group(1)

                # Shouldn't happen.
                if (labname == existingLab):
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc detach xdc.umdsectc"')
                    display(HTML("<span>Attaching the current lab.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc attach xdc ' + materialPattern + '"')
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))
            runlab = subprocess.run('su - umdsectc -c "bash /home/runlab ' + labname + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))

# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/umdsectc/notebooks/saves/umdsectc_sqli.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                    output0_2.clear_output()
                    display(HTML("<span style='color: green;'>Your lab has been successfully loaded.</span>"))
            elif (result.returncode == 2):
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>The sqli lab is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: SQL Basics</strong>

For the first topic, you are going to learn the basics of SQL, such as creating a database, a table, and practicing queries like adding, updating, and deleting entries from the table.

Before practicing SQL, some important background and setup. SQL is a relational database management system (DBMS), meaning that data is stored as tables. Non-relational DBMS's exist, which is where data are stored as document files. An example of a non-relational DBMS is MongoDB. There are a few different variations of SQL which you may find online. A couple that you may find are MySQL, SQLite, PostgreSQL, SQL Server, and MariaDB. These all use SQL, but may have feature and performance differences. 

This lab is going to use MariaDB, which is backwards compatible with MySQL. This means that applications that use MySQL can easily switch to MariaDB without losing performance or features. MariaDB is designed to have more features, better performance, and fewer bugs than MySQL, and it's also more scalable and has faster query speeds. MariaDB has <a href="https://www.geeksforgeeks.org/difference-between-mysql-and-mariadb/">several extra features</a>, such as ```CREATE OR REPLACE TABLE``` and JSON query support. These features are advanced, and will not be covered in the lab.

You may view documentation called <a href="https://mariadb.com/kb/en/incompatibilities-and-feature-differences-between-mariadb-10-5-and-mysql-8-/">Incompatibilities and Feature Differences Between MariaDB 10.5 and MySQL 8.0</a> if you would like to learn more about MariaDB and how it's different from MySQL.

### Step 1: Create Your First Database

MariaDB is already installed on your system, so you may access it by typing ```mariadb``` to open the interface. If you are more comfortable with MySQL, typing ```mysql``` will also bring up MariaDB since MariaDB is backwards-compatible, as mentioned before.

SQL statements are usually typed in UPPERCASE LETTERS, but is not required. However, to follow traditional syntax, queries will be written in UPPERCASE. Additionally, SQL statements end with a semicolon.

First, list your databases by typing ```SHOW DATABASES;```. Currently, there are a couple databases, where one is called ```fccu```. <u>This is required for later in the lab, so at the moment, you will need to create another database.</u>

Use ```CREATE DATABASE practice;``` to create a database.

In [4]:
# Click the button below to check your work.
step1Complete = False

# Function to check the permissions.
def step_1():
    # Required to change boolean value.
    global step1Complete

    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/section_1.py 1 NA', shell=True)

    if (result.returncode == 0):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True

    elif (result.returncode == 1):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>No database named \"practice\" was found. Try again.</span>"))
            step1Complete = False
    
    elif (result.returncode == 2):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>An error occurred when checking your database. Please contact your professor or TA.</span>"))
            step1Complete = False

def check_step_1(b):
    step_1()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check For Database")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check For Database', style=ButtonStyle())

Output()

### Step 2: Create Your First Table

You have created a database, and now you will need to "use" it. To select the database that you want to use, type ```USE practice;```. After using the ```USE``` command, you will need to create a table. Databases can have multiple tables to them, but you will only use one table for this lab.

We are going to create a table called ```students``` which will have three columns to them: ```student_id```, ```student_name```, and ```student_grade```. Each attribute has a type to them, similar to programming languages like C. Additionally, it's common for tables to have keys to them, which are unique identifiers for each entry. There are multiple types of keys, but we are going to use a ```PRIMARY KEY``` for the ```student_id``` attribute. A ```PRIMARY KEY``` is unique and cannot be a duplicate value. There are also candidate keys, super keys, alternate keys, foreign keys, and composite keys.

This is the layout of the table that you will create:
- ```student_id``` is of type ```SMALLINT UNSIGNED``` and a ```PRIMARY KEY```.
  - ```SMALLINT``` ranges between -32,768 to 32,767.
  - ```UNSIGNED``` takes the size of ```SMALLINT```, but only consists of positive values. Therefore, it will range from 0 to 65,535. We are using ```UNSIGNED``` so that there are no negative ID values.
  - <u>Note</u>: ```UNSIGNED``` is unsupported in PostgreSQL.
- ```student_name``` is of type ```VARCHAR(64)```
  - ```VARCHAR``` means "VARiable CHARacter", which dynamically allocates alphanumeric characters up to 64 characters.
  - This allows ```student_name``` to be 10 bytes in length if it's 10 characters long, up to 64 characters.
- ```student_grade``` is of type ```CHAR(1)```
  - ```CHAR``` is a fixed length of characters.
  - ```CHAR(15)``` will always take 15 bytes in memory, even if the string is not 15 characters long itself, like the word "database".
  - ```CHAR(1)``` is being used since students will be allocated a grade of A, B, C, D, or F.

The ```CREATE TABLE``` query can be lengthy, depending on the table. It consists of the following syntax: ```CREATE TABLE table_name (column1 datatype, column2 datatype, ...);```

Given the following columns and datatypes, create a table called ```students``` that has the following:
- ```student_id SMALLINT UNSIGNED PRIMARY KEY```
- ```student_name VARCHAR(64)```
- ```student_grade CHAR(1)```

In [5]:
# Click the button below to check your work.
step2Complete = False

# Function to check the permissions.
def step_2():
    # Required to change boolean value.
    global step2Complete

    with output2:
        output2.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/section_1.py 2 NA', shell=True)

    if (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True

    elif (result.returncode == 0):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>There was no table found named \"students\" inside of the \"practice\" database.</span>"))
            step2Complete = False
    
    elif (result.returncode == 2):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
            step2Complete = False

def check_step_2(b):
    step_2()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check For Table")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check For Table', style=ButtonStyle())

Output()

### Step 3: Inserting SQL Entries

To add SQL entries into the table, use the ```INSERT INTO``` command. The syntax is: ```INSERT INTO table (column1, column2, ...) VALUES (A, B, ...);```

These are the following entries that you will need to add into the table:
- ID: 100, Name: Aaron, Grade: A
- ID: 101, Name: Danny, Grade: B
- ID: 102, Name: Hannah, Grade: D
- ID: 103, Name: Abby, Grade: NULL

The order of the ```column1, column2, ...``` attributes must match the type of the ```VALUES```. For example, inserting an ```INT``` datatype does not require quotes, but ```CHAR``` or ```VARCHAR``` require quotes around them (similar to strings in programming languages).

One of the entries contains a ```NULL``` value. SQL allows ```NULL``` values, which are not equal to ```0``` or ```"NULL"```. NULL is not a string, it's a datatype.

Add the four entries to the ```students``` table.

<u>Tips<u>:
- Order does not matter when using ```INSERT INTO```. If you created your table as ```student_id```, ```student_grade```, then ```student_name```, you may use ```INSERT INTO students (student_id, student_name, student_grade), VALUES (...)```, where ```student_name``` and ```student_grade``` are in opposite order.
- Not all DBMS's support double quotes. Some SQL modes only support single quotes. Fortunately, MariaDB supports both single quotes and double quotes. Backticks (``` ` ```) can be used to identify columns, though this is optional. However, it may be required if you make a (poor) decision naming an attribute after a reserved keyword, like ``` `column` ```.
- As emphasized before, ```NULL``` is not a string. Consider this when adding your fourth entry.

In [6]:
# Click the button below to check your work.
step3Complete = False

# Function to check the permissions.
def step_3():
    # Required to change boolean value.
    global step3Complete

    with output3:
        output3.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/section_1.py 3 NA', shell=True)

    if (result.returncode == 1):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

    elif (result.returncode == 0):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>The four rows that were provided either cannot be found or have an incorrect value. Ensure that the only rows in your database are the ones listed above.</span>"))
            step3Complete = False
    
    elif (result.returncode == 2):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
            step3Complete = False

def check_step_3(b):
    step_3()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check For Table")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check For Table', style=ButtonStyle())

Output()

### Step 4: Practicing SQL Queries (Part 1)

You now have a table that's populated with entries. Now, you are going to perform some SQL statements that will return entries based on boolean values.

This is an example of a SQL query: ```SELECT * FROM table WHERE condition```

- ```SELECT *``` returns all columns. You are able to select certain columns of a table, like ```SELECT student_id```. 
- The ```FROM``` clause selects the table that you wish to grab entries from.
- The ```WHERE``` clause takes a condition that gives entries that match it.

Using the text prompt below, type a SQL statement that will print everything in the table. You do not need to use a ```WHERE``` clause for your query.

<u>Tip</u>: You are encouraged to try running this in MariaDB in your ```sqli``` node first. When your answer is ready, you may copy your SQL query into the entry box.

<strong>The remaining steps throughout this section will contain a "Reset" button.</strong> If you made an error to your database, and wish to revert your database to the previous step, click on the "Reset" button.

In [14]:
# Click the button below to check your work.
step4Complete = False

# Function to check if the student's answer was correct.
def step_4():
    # Required to change boolean values.
    global step4Complete

    # Loading, in case the check is slow.
    with output4:
        output4.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput4.value == ""):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step4Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput4.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "/home/.checker/section_1.py 4 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_4_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step4Complete = True
        
        elif (result.returncode == 0):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step4Complete = False
        
        elif (result.returncode == 2):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step4Complete = False

def check_step_4(b):
    step_4()

def reset_step_4(b):
    if (not step3Complete or step4Complete):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/reset_db.py 4', shell=True, stdout=subprocess.DEVNULL)
    
        output4.clear_output()
        with output4:
            if (result.returncode == 1):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 0):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading4 = widgets.Output()
display(loading4)
with loading4:
    loading4.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput4 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "cat /home/.checker/responses/step_4_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput4.value = result.stdout

# After the student's response was loaded, clear the output.
loading4.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_4 = widgets.Button(description="Reset Database")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)
button_step_4.on_click(reset_step_4)

# Display the output.
display(userInput4, button, button_step_4, output4)

Output()

Text(value='SELECT * FROM students;\n', description='Query:', layout=Layout(width='90%'), placeholder='Type yo…

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 5: Practicing SQL Queries (Part 2)

Using the ```WHERE``` clause, type a SQL statement that will return all students that have an "A" for a grade.

<u>Tip</u>: Your ```WHERE``` statement must consist of matching one of your columns with a ```VARCHAR``` (a string). Recall that comparing strings in SQL need "quotes" around them. You may use either single quotes or double quotes.

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check if the student's answer was correct.
def step_5():
    # Required to change boolean values.
    global step5Complete

    # Loading, in case the check is slow.
    with output5:
        output5.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput5.value == ""):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step5Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput5.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "/home/.checker/section_1.py 5 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_5_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step5Complete = True
        
        elif (result.returncode == 0):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step5Complete = False
        
        elif (result.returncode == 2):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step5Complete = False

def check_step_5(b):
    step_5()

def reset_step_5(b):
    if (not step4Complete):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/reset_db.py 5', shell=True, stdout=subprocess.DEVNULL)
    
        output5.clear_output()
        with output5:
            if (result.returncode == 1):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 0):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading5 = widgets.Output()
display(loading5)
with loading5:
    loading5.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput5 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "cat /home/.checker/responses/step_5_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput5.value = result.stdout

# After the student's response was loaded, clear the output.
loading5.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_5 = widgets.Button(description="Reset Database")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)
button_step_5.on_click(reset_step_5)

# Display the output.
display(userInput5, button, button_step_5, output5)

Output()

Text(value="SELECT * FROM students WHERE student_grade = 'A';\n", description='Query:', layout=Layout(width='9…

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 6: Practicing SQL Queries (Part 3)

Type a SQL statement that returns all students who do not have a ```NULL``` grade.

<u>Tip</u>: Recall that ```NULL``` is not a string, therefore it cannot be tested like the previous step. Instead, use ```IS NOT NULL``` in your ```WHERE``` statement.

In [9]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the student's answer was correct.
def step_6():
    # Required to change boolean values.
    global step6Complete

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput6.value == ""):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step6Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput6.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "/home/.checker/section_1.py 6 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_6_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step6Complete = True
        
        elif (result.returncode == 0):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step6Complete = False
        
        elif (result.returncode == 2):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step6Complete = False

def check_step_6(b):
    step_6()

def reset_step_6(b):
    if (not step5Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/reset_db.py 6', shell=True, stdout=subprocess.DEVNULL)
    
        output6.clear_output()
        with output6:
            if (result.returncode == 1):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 0):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput6 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput6.value = result.stdout

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_6 = widgets.Button(description="Reset Database")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)
button_step_6.on_click(reset_step_6)

# Display the output.
display(userInput6, button, button_step_6, output6)

Output()

Text(value='SELECT * FROM students WHERE student_grade IS NOT NULL;\n', description='Query:', layout=Layout(wi…

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 7: Practicing SQL Queries (Part 4)

SQL entries can be updated by using the ```UPDATE``` command.

The syntax works as follows: ```UPDATE table SET column1 = value1, column2 = value2, ... WHERE condition;```

The ```WHERE``` clause is also optional within the ```UPDATE``` command. However, when not matching a condition, it will change all values in a given column to a certain amount. Therefore, it's practical to use ```WHERE``` when working with the ```UPDATE``` command.

Type a SQL statement that changes Aaron's grade to a B.

<u>Tip</u>: If you need to revert your database back to what it originally was, you may click on the "Reset" button. This will add all entries back to your database again.

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check if the student's answer was correct.
def step_7():
    # Required to change boolean values.
    global step7Complete

    # Loading, in case the check is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput7.value == ""):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step7Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput7.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "/home/.checker/section_1.py 7 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_7_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step7Complete = True
        
        elif (result.returncode == 0):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step7Complete = False
        
        elif (result.returncode == 2):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step7Complete = False

def check_step_7(b):
    step_7()

def reset_step_7(b):
    if (not step6Complete):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/reset_db.py 7', shell=True, stdout=subprocess.DEVNULL)
    
        output7.clear_output()
        with output7:
            if (result.returncode == 1):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 0):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading7 = widgets.Output()
display(loading7)
with loading7:
    loading7.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput7 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "cat /home/.checker/responses/step_7_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput7.value = result.stdout

# After the student's response was loaded, clear the output.
loading7.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_7 = widgets.Button(description="Reset Database")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)
button_step_7.on_click(reset_step_7)

# Display the output.
display(userInput7, button, button_step_7, output7)

Output()

Text(value="UPDATE students SET student_grade = 'B' WHERE student_name = 'Aaron';\n", description='Query:', la…

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 8: Practicing SQL Queries (Part 5)

SQL entries can also be removed by using the ```DELETE``` command.

The syntax for ```DELETE``` is: ```DELETE FROM table WHERE condition;```

Type a SQL statement that removes everyone whose grade is NULL.

<u>Tip:</u> If you need to revert your database, you may click on the "Reset" button. Since you have completed Step 8, it will not reverse your progress from the previous step.

In [11]:
# Click the button below to check your work.
step8Complete = False

# Function to check if the student's answer was correct.
def step_8():
    # Required to change boolean values.
    global step8Complete

    # Loading, in case the check is slow.
    with output8:
        output8.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput8.value == ""):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step8Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput8.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "/home/.checker/section_1.py 8 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_8_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step8Complete = True
        
        elif (result.returncode == 0):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step8Complete = False
        
        elif (result.returncode == 2):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step8Complete = False

def check_step_8(b):
    step_8()

def reset_step_8(b):
    if (not step7Complete):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/reset_db.py 8', shell=True, stdout=subprocess.DEVNULL)
    
        output8.clear_output()
        with output8:
            if (result.returncode == 1):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 0):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading8 = widgets.Output()
display(loading8)
with loading8:
    loading8.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput8 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "cat /home/.checker/responses/step_8_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput8.value = result.stdout

# After the student's response was loaded, clear the output.
loading8.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_8 = widgets.Button(description="Reset Database")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)
button_step_8.on_click(reset_step_8)

# Display the output.
display(userInput8, button, button_step_8, output8)

Output()

Text(value='DELETE FROM students WHERE student_grade IS NULL;\n', description='Query:', layout=Layout(width='9…

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 9: Intermediate SQL Queries

Additional clauses may be used in SQL queries for more advanced searches. A couple of useful clauses are: ```LIMIT```, ```OFFSET```, ```ORDER BY```, and ```GROUP BY```. There are more clauses than this, but a couple of these clauses should be known when doing this lab.

- ```LIMIT``` takes the entries that your query returns, and only returns the top X rows, where X is the integer value from ```LIMIT X```.
- ```OFFSET``` is similar to ```LIMIT```, but skips the first X rows where X is the integer value from ```OFFSET X```.
  - ```OFFSET``` is useful when implementing previous page and next page functionality for websites.
  - ```OFFSET``` cannot be used without the ```LIMIT``` clause.
  - ```OFFSET``` cannot be called before ```LIMIT```.
- ```ORDER BY``` takes a column from your result query and sorts it numerically/alphabetically.
  - ```ASC``` and ```DESC``` are optional after the ```ORDER BY``` statement, which stand for "ascending" and "descending".
- ```GROUP BY``` is similar to ```ORDER BY```, except it is mainly used for aggregation functions, like ```COUNT()```, ```MAX()```, ```MIN()```, ```SUM()```, and ```AVG()```. These aggregation functions are placed in the ```SELECT``` statement of SQL queries, and return numerical values.
  - An example of ```GROUP BY``` is: ```SELECT COUNT(student_grade) FROM students WHERE student_grade == 'A' GROUP BY student_grade;```, which returns the number of students that have an A for a grade.
 
For your task, create a SQL statement that returns all entries in the table, but prints only the first row and offset your result by one.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check if the student's answer was correct.
def step_9():
    # Required to change boolean values.
    global step9Complete

    # Loading, in case the check is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput9.value == ""):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step9Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput9.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "/home/.checker/section_1.py 9 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_9_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step9Complete = True
        
        elif (result.returncode == 0):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step9Complete = False
        
        elif (result.returncode == 2):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step9Complete = False

def check_step_9(b):
    step_9()

def reset_step_9(b):
    if (not step8Complete):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@sqli /home/.checker/reset_db.py 9', shell=True, stdout=subprocess.DEVNULL)
    
        output9.clear_output()
        with output9:
            if (result.returncode == 1):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 0):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading9 = widgets.Output()
display(loading9)
with loading9:
    loading9.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput9 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@sqli "cat /home/.checker/responses/step_9_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput9.value = result.stdout

# After the student's response was loaded, clear the output.
loading9.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_9 = widgets.Button(description="Reset Database")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)
button_step_9.on_click(reset_step_9)

# Display the output.
display(userInput9, button, button_step_9, output9)

Output()

Text(value='SELECT * FROM students LIMIT 1 OFFSET 1;\n', description='Query:', layout=Layout(width='90%'), pla…

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

## <strong>Topic 2: PHP and SQL Practice</strong>

PHP, as mentioned before, is a programming language to handle server-side processing. It's used in back-end website development, which is everything that the website users cannot see. Front-end web development is everything that the users of a website can see, like HTML, CSS, and JavaScript.

Since PHP is back-end, it will hide SQL statements from the user, which provides some security for websites. However, when focusing on computer security, you cannot assume that the hacker doesn't know the system. For this topic, you will learn how SQL statements are called from PHP, which take user input from HTML forms. You are going to view a sample of unsafe PHP code that can put your database at risk. Additionally, you will learn how to break an insecure system.

<strong>This portion of the lab is going to require port forwarding, which requires setup on your own, local machine. If you have not already done port forwarding, you will need to follow the steps from this notebook.</strong>

### Step 10: Accessing ```php_practice.php```

After you have port forwarded to your ```sqli``` node, access ```localhost:port/cgi-bin/php_practice.php```, where ```port``` is the number that you used in your SSH statement.

<strong>For example</strong>, if you have already set up port forwarding, type the following command into your terminal: ```ssh -L 8080:sqli:80 umdsecXX-xdc-umdsecXX```, then access the practice page by typing ```localhost:8080/cgi-bin/php_practice.php``` into your web browser.

The website's source code is located at ```/usr/lib/cgi-bin/php_practice.php```.

This website contains a simple form which takes a student's ID. When you type in a student's ID into the field, it takes your input, puts it into a SQL statement, sends the SQL statement to the database, retrieve the results, then it displays it on the website.

For your convenience and to help demonstrate the lab, the SQL statement that's being sent to the database will display on the page for you, highlighting the statement that you typed in. Feel free to use this to assist your understanding of SQL injection, as it will be unavailable for the large scale portion of the lab.

### Step 11: Observe the PHP File

Now, open ```/usr/lib/cgi-bin/php_practice.php``` in your ```sqli``` node, and observe the file contents. Look for the SQL query, and observe how the user input is used in the SQL statement.

In the text field below, explain the possible security vulnerability with this approach.

### Step 12: Practicing SQL Injection (Part 1)

In the previous step, you should've observed the security vulnerability with this PHP code. From Step 10, you observed that this SQL statement works, but there is a major flaw with this design.

Using this to your advantage, type SQL code into the "Student ID" field on the website to change Aaron's name to Taylor.

After testing your statement, you will need to copy/paste your input into the field below to check your work. Your database will be reset and your query will be tested to determine if your SQL statement is correct. If you would like to reset your database, you may click on the "Reset" button on the website.

<u>Tip</u>: Notice that SQL statements end with a semicolon. Observe how your SQL statement gets inserted into the PHP string. In order for this injection attack to work, it must be a valid SQL statement. SQL statements can be concatenated into a single query by using semicolons.

### Step 13: Practicing SQL Injection (Part 2)

Now, using the input field on the website, change Hannah's grade to a B.

Once you have a working command, type the command below and check your work. Your database will be reverted back to completing Step 12, and it will be tested. Again, a "Reset" button is available for you, and it will not reset your progress from the previous step.

### Step 14: Practicing SQL Injection (Part 3)

Finally, using the input field, create a student named Jason whose ID is 200 and has an F for a grade.

Like the checks, your database will be reverted to the previous step, and it will be tested for correctness. A "Reset" button is available to revert to the previous step.

## <strong>Topic 3: Prepared Statements</strong>

PHP has a built-in functionality to mitigate the risks of SQL injection.

### mysqli_real_escape_string() - <em style="color: red;">Unsafe</em>

First, there is a function called ```mysqli_real_escape_string()```, which is an <strong>unsafe</strong> sanitization function. 

This function requires a connection to be made with the SQL server, which can provide undefined behavior if a connection isn't made before the function is called. This function takes any unsafe characters that can occur within a SQL statement, and safely escapes characters so that it treats SQL tokens as regular string characters. However, there are security vulnerabilities to this statement, which you can <a href="https://stackoverflow.com/questions/5741187/sql-injection-that-gets-around-mysql-real-escape-string">read about here</a>. 

To put simply, this function becomes unsafe if your database character set does not match your PHP character set. This means using double quotes for your SQL statement, but single quotes for your user input, or vice versa.

### mysqli_prepare() - <em style="color: green;">Safe</em>

The safer alternative of ```mysqli_real_escape_string()``` is ```mysqli_prepare()```, which is a <u>prepared statement</u>.

A prepared statement is a pre-compiled query, which means that it's sent to the database before it takes input. Once it has been compiled, the user input is replaced by ```?``` within the query, and becomes impossible to inject SQL code into. Prepared statements take the input type for each ```?``` within the query, and their types get checked when compiling the code before replacing the ```?``` with the user's input.

Here is an example of a prepared statement:

```$stmt = mysqli_prepare(conn, "INSERT INTO table (column1, column2) VALUES (?, ?)");```

Before a statement can be executed, you need to bind parameters to the ```?``` values in the prepared statement. This tells PHP which variables are being assigned to the question marks in the statement. So, the next line would be:

```mysqli_stmt_bind_param($stmt, 'ss', $userInput1, $userInput2);```

The ```ss``` tells SQL that ```$userInput1``` is a string, as well as ```$userInput2```. Additionally, if any inputs are ```i```, it tells SQL that you are sending an int, ```d``` is a float, and ```b``` is a blob. You only need to worry about ```i``` and ```s``` for this lab.

Finally, the last step is to run ```mysqli_stmt_execute($stmt);``` to execute the statement. <u>This is not specific to prepared statements, and is already written for you in the exercises.</u> If your SQL query returns a row from a table, it can be retrieved with ```$result = mysqli_stmt_get_result($stmt);```, then you can convert the result into an array with ```$row = mysqli_fetch_array($result);```. Place this statement within a ```while``` loop if you would like to read all rows from the query.

### Step 15: Making a Prepared Statement

Using the instructions above, turn the SQL statement in ```php_practice.php``` into a prepared statement. You are encouraged to take your payloads from above and try to alter the database through the input field.

If you would like to revert the database to completing Step 14, you may click on the reset button. This will not reset your PHP file.

When you have finished writing your prepared statement, click on the "Check Work" button to test this step. A SQL injection will be tested on your database, which should fail.

## <strong>Topic 4: A Large-Scale Application of SQL Injection</strong>

For the final topic, you will be testing your knowledge on a website called FrobozzCo Community Credit Union (FCCU). The developers for FCCU have written the database without knowledge about SQL injection, and you will be responsible for attacking their website, draining someone's savings, then fixing their website.

Navigate to ```localhost:port/index.html``` to go to the bank's landing page. Once again, <strong>port forwarding is required</strong> in order to access the website from your browser. ```port``` is a value that you chose in your SSH statement.

Earlier in the lab, you saw a database called ```fccu``` in MySQL. The website that you will break is doing to use this database. If you are currently signed into MySQL, type ```USE fccu``` to switch databases. Inside of the database, you will be able to view the usernames and passwords of all accounts. You may try signing into any account to view what customers can view.

For this topic, you will not be shown the SQL query upon submitting the forum. You will need to think critically and understand how your input will be placed within the SQL statements.

### Step 16: Accessing Someone Else's Account - CHECK LATER

On the sign-in page, type a SQL statement into the ID field so that you can access anyone's account, without knowing the ID numbers ahead of time.

When you have a working payload, copy/paste it below to check your work.

<u>Tips<u>:
- The ID field of the user's input is placed in the middle of a SQL statement. Consider using a SQL comment (```--```) to remove everything after your injection so that there is no error.
- Signing into the website works when a statement only returns one row. Consider using ```LIMIT``` within your statement. - CHECK LATER
- It's impossible to comment out a SQL statement before your payload. This means that ```id``` must be checked, since ```WHERE id = $id``` is the SQL statement that you're injecting into. You do not know anyone's ID, so consider using the ```OR``` clause within your payload, and provided a valid argument for the ```id = XXX``` within the statement. - CHECK LATER

### Step 17: Accessing Multiple Accounts

In the previous step, you found a way to access someone's account by using SQL injection. Now, on the sign-in page, type a SQL statement into the ID field so that you can access the next valid ID.

When you have a working payload, copy/paste it below to check your work.

<u>Tip</u>: This payload will be very similar to the previous one. However, consider using ```OFFSET```. (Also consider ID > something)

### Step 18: Wiping a Bank Account

Make some account (your choice) wire its total balance to the bank with routing number: 314159265 and account number: 271828182845

<em>This step will not be checked, but you can do this to view the danger of what your payload allowed you to do.</em> - (Create a log file of transfers and check to see if balances have updated.)

### Step 19: Adding/Updating Accounts

Explain why you can't create a new account or arbitrarily update account balances within the username field.

This is a multiple choice question:

a) <u>You cannot have more than one query in a statement when executing from PHP.</u> 

Correct! When not using prepared statements, the ```query()``` statement is being used. However, if you want to execute multiple queries within one statement, you need to use ```mysqli_multi_query()```. Source: https://www.php.net/manual/en/mysqli.multi-query.php

b) <u>A ```SELECT``` statement cannot be nested with a ```INSERT``` or ```UPDATE``` statement.</u> 

Incorrect. Multiple queries can be called together, no matter their behavior. They can be separated with semicolons, and it's still a valid SQL query.

c) <u>```fccu.php``` needs to read output from the SQL statement, and ```INSERT``` or ```UPDATE``` provides no output. So, the statement(s) cannot be used.</u>

Incorrect. Although ```INSERT``` and ```UPDATE``` return zero rows, it will still be a valid query. If such an attack worked, returning zero rows would likely produce a "User Not Found" error, but insertion/updating would still happen.

d) <u>The ID field restricts the number of characters you can type, and doing an insertion would max out the character limit.</u>

Incorrect. The ID field allows an unlimited amount of characters. However, not restricting the ID field is a security vulnerability, as well as allowing characters instead of just numbers.

### Step 20: Fixing the Vulnerability

Convert the ID field that you broke into a prepared statement. Click on the button to test your work. If the test passes, then your prepared statement works and you will have finished the lab.

- Log in page
- Transfer page
- Anywhere that user input is possible

### NOTES FOR RUNLAB:

You will need to run these for them to work:
- sudo apt install python3-pip
- pip install mysql-connector-python